### Find your API Key at https://quantum-computing.ibm.com/

### Follow the eBook at https://qiskit.org/textbook/preface.html

A qubit is like a 'quantum coin' that has two possible states: 0 and 1. Except they use quantities called amplitudes. </br>
So the probability of a qubit being 1 or 0 is still 50%, it's just because sqrt(0.5)^2 = 50% where sqrt(0.5) is the amplitude.</br>
Negative amplitudes cancel out positve ones to return 0 probabilities. This is an interference effect that can be used to  combine operations to build more efficient algorithms. These algorithms can use interference effects to make the wrong answers cancel out quickly and give us a high probability of measuring the right answer. This is the idea behind quantum computing.

In [10]:
from qiskit import QuantumCircuit, assemble, Aer
from qiskit.visualization import plot_histogram
from matplotlib import pyplot as plt

### The elements of working with a simple quantum circuit are:
* Encode Input
* Execute Algorithm
* Extract the Result 

In [7]:
# First encode the input
# This cell creates a circuit with 8 qubits and 8 outputs

n = 8
n_q = n # number of qubits in circuit
n_b = n # number of output bits

# QuantumCircuit() maps the input to the output bits
qc_output = QuantumCircuit(n_q,n_b)

In [8]:
# Qubits are always initialized at 0
# extraction of outputs is done through measure
# measure takes a qubit, and maps it to an output bit

for j in range(n):
    qc_output.measure(j,j)
    
# the draw() function draws a quantum circuit diagram
qc_output.draw()

┌─┐                     
q_0: ┤M├─────────────────────
     └╥┘┌─┐                  
q_1: ─╫─┤M├──────────────────
      ║ └╥┘┌─┐               
q_2: ─╫──╫─┤M├───────────────
      ║  ║ └╥┘┌─┐            
q_3: ─╫──╫──╫─┤M├────────────
      ║  ║  ║ └╥┘┌─┐         
q_4: ─╫──╫──╫──╫─┤M├─────────
      ║  ║  ║  ║ └╥┘┌─┐      
q_5: ─╫──╫──╫──╫──╫─┤M├──────
      ║  ║  ║  ║  ║ └╥┘┌─┐   
q_6: ─╫──╫──╫──╫──╫──╫─┤M├───
      ║  ║  ║  ║  ║  ║ └╥┘┌─┐
q_7: ─╫──╫──╫──╫──╫──╫──╫─┤M├
      ║  ║  ║  ║  ║  ║  ║ └╥┘
c: 8/═╩══╩══╩══╩══╩══╩══╩══╩═
      0  1  2  3  4  5  6  7

In [39]:
# This is the simulator we'll use to simulate a quantum calculation
sim = Aer.get_backend('qasm_simulator')  
qobj = assemble(qc_output)  # this turns the circuit into an object our backend can run
result = sim.run(qobj).result()  # we run the experiment and get the result from that experiment

# from the results, we get a dictionary containing the number of times (counts) each result appeared
counts = result.get_counts()
# because these bits were all initialized as 0, th output is all 0s
counts

{'00000000': 1024}

### Gates Encode Qubits 

The x() operation flips the qubit from the default 0 to 1, acting as a NOT gate.

In [40]:
qc_encode = QuantumCircuit(n)

# This flips the 7th Qubit to 1
qc_encode.x(7)
qc = qc_encode + qc_output
qc.draw()

┌─┐                     
q_0: ─────┤M├─────────────────────
          └╥┘┌─┐                  
q_1: ──────╫─┤M├──────────────────
           ║ └╥┘┌─┐               
q_2: ──────╫──╫─┤M├───────────────
           ║  ║ └╥┘┌─┐            
q_3: ──────╫──╫──╫─┤M├────────────
           ║  ║  ║ └╥┘┌─┐         
q_4: ──────╫──╫──╫──╫─┤M├─────────
           ║  ║  ║  ║ └╥┘┌─┐      
q_5: ──────╫──╫──╫──╫──╫─┤M├──────
           ║  ║  ║  ║  ║ └╥┘┌─┐   
q_6: ──────╫──╫──╫──╫──╫──╫─┤M├───
     ┌───┐ ║  ║  ║  ║  ║  ║ └╥┘┌─┐
q_7: ┤ X ├─╫──╫──╫──╫──╫──╫──╫─┤M├
     └───┘ ║  ║  ║  ║  ║  ║  ║ └╥┘
c: 8/══════╩══╩══╩══╩══╩══╩══╩══╩═
           0  1  2  3  4  5  6  7

In [42]:
# Qubits are assigned right to left
# Assembling the circuit now returns a 1 at the 7th position

qobj = assemble(qc)
counts = sim.run(qobj).result().get_counts()
counts

{'10000000': 1024}

In [32]:
# Use x() to flip bits from 1 to 0 in our circuit

qc_encode = QuantumCircuit(n)
qc_encode.x(1)
qc_encode.x(5)

qc = qc_encode + qc_output
qobj = assemble(qc)
counts = sim.run(qobj).result().get_counts()
counts

{'00100010': 1024}

## Half - Adder 

The previous cells just described a circuit that had no algorithm, just a mapping from initial state to output. </br>
Now we will build a circuit capable of doing calculations. </br>
To do this we'll need two new gates: CNOT and Toffoli or XOR and AND logic gates. </br>
The NOT, CNOT and Tofoli gates allow us to add any set of numbers together.

#### CNOT Gate 

The CNOT gate can tell us if the output bits are the same or different.

In [98]:
# We can use cx() function to encode the CNOT gate
qc_cnot = QuantumCircuit(2)
qc_cnot.cx(0,1)

# Here q_0 is the control qubit, while q_1 is the target qubit. Together they form the CNOT gate
qc_cnot.draw()

q_0: ──■──
     ┌─┴─┐
q_1: ┤ X ├
     └───┘

In [99]:
# Both qubits are initialized as 0, so they are the same, outputting 0s

qc = QuantumCircuit(2,2)
qc.cx(0,1)
qc.measure(0,1)
qc.measure(1,1)
qobj = assemble(qc)
counts = sim.run(qobj).result().get_counts()
print(counts)

{'00': 1024}


In [95]:
qc = QuantumCircuit(2,2)
qc.x(0) # Adding this x() changes one of the bits so they are different, outputting a 1
qc.cx(0,1)
qc.measure(0,1)
qc.measure(1,1)
qobj = assemble(qc)
counts = sim.run(qobj).result().get_counts()
print(counts)

{'10': 1024}


In [96]:
qc = QuantumCircuit(2,2)
qc.x(0) # Adding this x() changes one of the bits so they are different
qc.x(1) # Adding this second x() makes them the same again, outputting 0s
qc.cx(0,1)
qc.measure(0,1)
qc.measure(1,1)
qobj = assemble(qc)
counts = sim.run(qobj).result().get_counts()
print(counts)

{'00': 1024}


In [107]:
# Initialize a 4-qubit quantum circuit
qc_ha = QuantumCircuit(4,2)

# encode inputs in qubits 0 and 1 so they are different
qc_ha.x(0) # For a=0, remove this line. For a=1, leave it.
qc_ha.x(1) # For b=0, remove this line. For b=1, leave it.

# The barrier() function separates the different parts of the circuit
qc_ha.barrier()

# use cnots to write the XOR of the inputs on qubit 2
qc_ha.cx(0,2)
qc_ha.cx(1,2)
qc_ha.barrier()

# extract outputs
qc_ha.measure(2,0) # extract XOR value
qc_ha.measure(3,1)

qc_ha.draw()

░            ░       
q_0: ──────░───■────────░───────
     ┌───┐ ░   │        ░       
q_1: ┤ X ├─░───┼────■───░───────
     └───┘ ░ ┌─┴─┐┌─┴─┐ ░ ┌─┐   
q_2: ──────░─┤ X ├┤ X ├─░─┤M├───
           ░ └───┘└───┘ ░ └╥┘┌─┐
q_3: ──────░────────────░──╫─┤M├
           ░            ░  ║ └╥┘
c: 2/══════════════════════╩══╩═
                           0  1

In [112]:
qobj = assemble(qc_ha)
counts = sim.run(qobj).result().get_counts()
print(counts)

{'01': 1024}


### Toffoli gate
This is the equivalent of the AND boolean gate. It measures whether both bits are 1.

In [113]:
qc_ha = QuantumCircuit(4,2)
# encode inputs in qubits 0 and 1
qc_ha.x(0) # For a=0, remove the this line. For a=1, leave it.
qc_ha.x(1) # For b=0, remove the this line. For b=1, leave it.
qc_ha.barrier()
# use cnots to write the XOR of the inputs on qubit 2
qc_ha.cx(0,2)
qc_ha.cx(1,2)
# use ccx to write the AND of the inputs on qubit 3
qc_ha.ccx(0,1,3)
qc_ha.barrier()
# extract outputs
qc_ha.measure(2,0) # extract XOR value
qc_ha.measure(3,1) # extract AND value

qc_ha.draw()

┌───┐ ░                 ░       
q_0: ┤ X ├─░───■─────────■───░───────
     ├───┤ ░   │         │   ░       
q_1: ┤ X ├─░───┼────■────■───░───────
     └───┘ ░ ┌─┴─┐┌─┴─┐  │   ░ ┌─┐   
q_2: ──────░─┤ X ├┤ X ├──┼───░─┤M├───
           ░ └───┘└───┘┌─┴─┐ ░ └╥┘┌─┐
q_3: ──────░───────────┤ X ├─░──╫─┤M├
           ░           └───┘ ░  ║ └╥┘
c: 2/═══════════════════════════╩══╩═
                                0  1

In [114]:
qobj = assemble(qc_ha)
counts = sim.run(qobj).result().get_counts()
print(counts)

{'10': 1024}
